<a href="https://colab.research.google.com/github/tkbhms84/FaceMask/blob/main/FaceMaskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [ ]:
import cv2

In [ ]:
categories = ['with_mask', 'without_mask']

In [ ]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def detect_face(img):
    coords = haar.detectMultiScale(img)
    return coords

In [ ]:
def augment_image(img):
    # Flip horizontally with 50% probability
    if random.random() > 0.5:
        img = cv2.flip(img, 1)

    # Random rotation (-15 to 15 degrees)
    angle = random.randint(-15, 15)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))

    # Random brightness adjustment
    factor = 0.5 + random.uniform(0, 1)  # Between 0.5 (darker) and 1.5 (brighter)
    img = np.clip(img * factor, 0, 255).astype(np.uint8)

    # Random Gaussian noise
    if random.random() > 0.7:  # Apply noise with 30% probability
        noise = np.random.normal(0, 15, img.shape).astype(np.uint8)
        img = cv2.add(img, noise)

    return img


In [ ]:
data=[]
for category in categories:

  path = os.path.join('train',category)
  label = categories.index(category)

  for file in os.listdir(path):
    img_path = os.path.join(path,file)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Warning: Unable to read {img_path}")
        continue  # Skip this file
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = detect_face(gray)

    # Crop the first detected face
    if len(faces) > 0:
        x, y, w, h = faces[0]  # Get coordinates of the first detected face
        img = img[y:y+h, x:x+w]  # Crop the face from the image
    else:
        print(f"Warning: Unable to read {img_path}")
        continue  # Skip this file

    img = cv2.resize(img,(224,224))
    # # Apply data augmentation
    # aug_img = augment_image(img)
    data.append([img,label])
    # data.append([aug_img, label])

In [ ]:
len(data)

In [ ]:
import random
random.shuffle(data)

In [ ]:
X = []
y = []
for features,label in data:
  X.append(features)
  y.append(label)

len(X)
len(y)

In [ ]:
import numpy as np

In [ ]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)
X=X/255

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16()
vgg.summary()

In [ ]:
from keras import Sequential
model = Sequential()

In [ ]:
for layer in vgg.layers[:-1]:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False

model.summary()

In [ ]:
from keras.layers import Dense
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# Suppose this is your training history
history = model.fit(X_train, y_train, epochs=7, validation_data=(X_test, y_test))

# Plotting
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# model save
import joblib
filename = 'model2.sav'
joblib.dump(model, filename)

In [ ]:
# model load
import joblib

filename = 'model2.sav'
mymodel = joblib.load(filename)

In [ ]:
def detect_face_mask(img):
    y_pred = mymodel.predict(img.reshape(1, 224, 224, 3))
    return 1 if y_pred[0][0] > 0.5 else 0  # Binary classification threshold

In [ ]:
def draw_label(img,text,pos,bg_color):

  text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)

  end_x = pos[0] + text_size[0][0] + 2
  end_y = pos[1] + text_size[0][1] - 2

  cv2.rectangle(img,pos, (end_x,end_y),bg_color,cv2.FILLED)
  cv2.putText(img,text, pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret or frame is None:
        print("Failed to capture frame. Retrying...")
        continue  # Skip the iteration if frame is empty

    img = frame
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = detect_face(gray)

    # Crop the first detected face
    if len(faces) > 0:
        x, y, w, h = faces[0]  # Get coordinates of the first detected face
        img = img[y:y+h, x:x+w]  # Crop the face from the image

    img = cv2.resize(img, (224, 224))

    y_pred = detect_face_mask(img)
    coods = faces

    for x, y, w, h in coods:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 3)

    if y_pred == 0:
        draw_label(frame, "Mask", (30, 30), (0, 255, 0))
    else:
        draw_label(frame, "No Mask", (30, 30), (0, 0, 255))

    cv2.imshow("window", frame)

    # Check for 'q' key press to safely exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources properly after the loop
cap.release()
cv2.destroyAllWindows()